# Newton Divided Difference Interpolation - GUI Tool
## Syed Aashir Majeed - 10173
## Muqaddas Ashraf - 9123

## Importing libraries and modules 

In [1]:
import pandas as pd
from tkinter import *
from tkinter import ttk
from datetime import datetime
from tkinter import messagebox

## Import Dataset of Coronavirus cases

In [2]:
#Import dataset
dataset = pd.read_csv('owid-covid-data.csv').values.tolist()

country_names = pd.read_csv('owid-covid-data.csv')['location'].unique().tolist()

## Functions

## 1) Generate dataset accoring to the country

In [3]:
country_dataset = []
point_of_interpolation = 0

def generate_dataset():
    try:
        global country_dataset
        
        country_name = country_selected.get()
        country_dataset = []
        for i in range(len(dataset)):
            if dataset[i][2] == country_name:
                date_string = dataset[i][3]
                date_obj = datetime.strptime(date_string, '%Y-%m-%d')
                date_of_the_year = date_obj.timetuple().tm_yday
                try:
                    num_of_cases = int(dataset[i][4])
                except:
                    continue
                country_dataset.append([date_of_the_year, date_string, num_of_cases])

        #generate dataframe to show
        country_dataset_df = pd.DataFrame(country_dataset)
        country_dataset_df.columns = ['Day', 'Date', 'Cases']
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            display(country_dataset_df)
            
        #disable 
        country_input.config(state='disabled')
        confirm_country.config(state='disabled')
        
        #enable
        max_range = len(country_dataset)
        num_of_data_points.set(str(max_range))
        num_input.config(state='normal')
        confirm_num.config(state='normal')

        #show message
        messagebox.showinfo('Dataset Generated', 'Dataset was generated successfully')
    except Exception as e:
        messagebox.showerror('Error', e)

## 2) Limit number of data points

In [4]:
def limit_datapoints():
    try:
        try:
            global country_dataset
            
            limit_of_datapoints = int(num_of_data_points.get())
            if limit_of_datapoints > 0 and limit_of_datapoints <= len(country_dataset):
                country_dataset = country_dataset[-limit_of_datapoints:]
                
                #generate dataframe to show
                country_dataset_df = pd.DataFrame(country_dataset)
                country_dataset_df.columns = ['Day', 'Date', 'Cases']
                with pd.option_context('display.max_rows', None, 'display.max_columns', None):
                    display(country_dataset_df)
                    
                #disable 
                num_input.config(state='disabled')
                confirm_num.config(state='disabled')

                #enable
                point_input.config(state='normal')
                confirm_point.config(state='normal')
                
                #show message
                messagebox.showinfo('Dataset Confirned', f'Dataset was limited to {limit_of_datapoints} data points')
            else:
                messagebox.showerror('Error', f'Please input a number greater than 0 or less than or equal to {len(country_dataset)}')
        except Exception as e:
            messagebox.showerror('Error', f'Please input a number greater than 0 or less than or equal to {len(country_dataset)}')
    except Exception as e:
        messagebox.showerror('Error', e)

## 3) Calculate and implement Newton's formula

In [5]:
def calculate():
    try:
        try:
            global point_of_interpolation
            point_of_interpolation = float(interpolation_point.get()) 
            
            #show message
            messagebox.showinfo('Calculation Started', 'Calculating value for the entered interpolation point')
            root.destroy()
            
            #call newton divided difference interpolation function
            newton_divided_difference()
        except Exception as e:
            messagebox.showerror('Error', 'Please input a valid float or an integer')
    except Exception as e:
        messagebox.showerror('Error', e)
        
def newton_divided_difference():
    try:
        global country_dataset
        global point_of_interpolation
        
        # Newton divided difference formula 

        # Function to find the product term 
        def proterm(i, value, x): 
            pro = 1
            for j in range(i): 
                pro = pro * (value - x[j])
            return pro

        # Function for calculating 
        # divided difference table 
        def dividedDiffTable(x, y, n): 
            for i in range(1, n): 
                for j in range(n - i): 
                    y[j][i] = ((y[j][i - 1] - y[j + 1][i - 1]) / (x[j] - x[i + j]))
                return y; 

        # Function for applying Newton's 
        # divided difference formula 
        def applyFormula(value, x, y, n):
            sum = y[0][0]; 
            for i in range(1, n): 
                sum = sum + (proterm(i, value, x) * y[0][i]); 
            return sum; 

        # Function for displaying divided 
        # difference table 
        def printDiffTable(y, n): 
            for i in range(n): 
                for j in range(n - i): 
                    print(round(y[i][j], 4), "\t", end = " "); 
                print(""); 

        # number of inputs given 
        n = len(country_dataset)
        y = [ [0 for i in range(n)] for j in range(n)]

        x = []
        for i in range(len(country_dataset)):
            x.append(country_dataset[i][0])

        y_temp = []
        for i in range(len(country_dataset)):
            y_temp.append(country_dataset[i][2])

        for i in range(len(y_temp)):
            y[i][0] = y_temp[i]

        # calculating divided difference table 
        y=dividedDiffTable(x, y, n)

        # displaying divided difference table 
        printDiffTable(y, n)

        # value to be interpolated 
        value = point_of_interpolation

        # printing the value 
        print("\nValue at", value, "is", round(applyFormula(value, x, y, n), 2))
    except Exception as e:
        print(e)

## GUI Coding

In [7]:
root = Tk()
root.resizable(False, False)
root.title('Interpolation Tool for Coronavirus Dataset')
ws = root.winfo_screenwidth()
hs = root.winfo_screenheight()
w = 992
h = 550
x = (ws/2) - (w/2)
y = (hs/2) - (h/2)
root.geometry('%dx%d+%d+%d' % (w, h, x, y))

top_frame = Frame(root, width='992', height='90')
top_frame.pack(side=TOP)
top_frame.pack_propagate(0)

lbl_heading = Label(top_frame, font=('Arial', 24), text='Interpolation Tool for Coronavirus Dataset', bg='#015668', fg='#fff', height=90)
lbl_heading.config(anchor=CENTER)
lbl_heading.pack(side=TOP, fill=BOTH)

content_frame = Frame(root, width='992', height='460', bg='#eee')
content_frame.pack(side=TOP)
content_frame.pack_propagate(0)

step1_label = Label(content_frame, font=('Arial', 16), text='1. Generate dataset for input', padx=25, pady=20, bg='#eee')
step1_label.config(anchor='w')
step1_label.pack(side=TOP, fill=BOTH)

country_selected = StringVar()

country_input = ttk.Combobox(content_frame, state='readonly', values=country_names, font=('Arial', 12), textvariable=country_selected)
country_input.current(1)
country_input.pack(side=TOP, padx=(25, 750), fill=BOTH)

confirm_country = Button(content_frame, text='GENERATE', font=('Arial', 12), bg='#015668', fg='#fff',  disabledforeground="white", command=generate_dataset)
confirm_country.pack(side=TOP, padx=(25, 750), pady=(10,0), fill=BOTH)

step2_label = Label(content_frame, font=('Arial', 16), text='2. Input number of data points to include', padx=25, pady=20, bg='#eee')
step2_label.config(anchor='w')
step2_label.pack(side=TOP, fill=BOTH)

num_of_data_points = StringVar()

num_input = Entry(content_frame, font=('Arial', 12), state='disabled', textvariable=num_of_data_points)
num_input.pack(side=TOP, padx=(25, 750), fill=BOTH)

confirm_num = Button(content_frame, text='CONFIRM', font=('Arial', 12), bg='#015668', fg='#fff', state='disabled', disabledforeground="white", command=limit_datapoints)
confirm_num.pack(side=TOP, padx=(25, 750), pady=(10,0), fill=BOTH)

step3_label = Label(content_frame, font=('Arial', 16), text='3. Input interpolation point', padx=25, pady=20, bg='#eee')
step3_label.config(anchor='w')
step3_label.pack(side=TOP, fill=BOTH)

interpolation_point = StringVar()

point_input = Entry(content_frame, font=('Arial', 12), state='disabled', textvariable=interpolation_point)
point_input.pack(side=TOP, padx=(25, 750), fill=BOTH)

confirm_point = Button(content_frame, text='CALCULATE', font=('Arial', 12), bg='#015668', fg='#fff', state='disabled', disabledforeground="white", command=calculate)
confirm_point.pack(side=TOP, padx=(25, 750), pady=(10,0), fill=BOTH)

root.mainloop()

,Day,Date,Cases
0,29,2020-01-29,4
1,30,2020-01-30,4
2,31,2020-01-31,4
3,32,2020-02-01,4
4,33,2020-02-02,5
5,34,2020-02-03,5
6,35,2020-02-04,5
7,36,2020-02-05,5
8,37,2020-02-06,5
9,38,2020-02-07,5


,Day,Date,Cases
0,326,2020-11-21,157785
1,327,2020-11-22,158990
2,328,2020-11-23,160055
3,329,2020-11-24,161365
4,330,2020-11-25,162662
5,331,2020-11-26,163967
6,332,2020-11-27,165250
7,333,2020-11-28,166502
8,334,2020-11-29,167753
9,335,2020-11-30,168860


157785 	 1205.0 	 0 	 0 	 0 	 0 	 0 	 0 	 0 	 0 	 
158990 	 1065.0 	 0 	 0 	 0 	 0 	 0 	 0 	 0 	 
160055 	 1310.0 	 0 	 0 	 0 	 0 	 0 	 0 	 
161365 	 1297.0 	 0 	 0 	 0 	 0 	 0 	 
162662 	 1305.0 	 0 	 0 	 0 	 0 	 
163967 	 1283.0 	 0 	 0 	 0 	 
165250 	 1252.0 	 0 	 0 	 
166502 	 1251.0 	 0 	 
167753 	 1107.0 	 
168860 	 

Value at 500.0 is 367455.0
